In [1]:
%matplotlib inline
import copy
from qutip import *
import numpy as np
import math, cmath
import matplotlib.pyplot as plt
from tqdm import tqdm

from stateobj import Physics

import os

from IPython.display import Markdown, display

In [2]:
TIMESTEPS = 5000
TIMEDELTA = 1e-01
OMEGA = 0.5  # Strength of Interaction

D = 20

p = Physics(dimension=D, interaction_strength=OMEGA, interaction_time=TIMEDELTA)

th = OMEGA * TIMEDELTA
alpha = 0.1
phi = np.pi/2
ga = 2*alpha**2
gb = (1-alpha)**2*(1-np.cos(phi))

### Bosonic Operators
$$
\hat{C} = \cos(\theta\sqrt{2\hat{a}\hat{a}^\dagger}), \quad
    \hat{C}' = \cos(\theta\sqrt{2\hat{a}^\dagger\hat{a}}), \quad
    \hat{S} = \hat{a}^\dagger\frac{\sin(\theta\sqrt{2\hat{a}\hat{a}^\dagger})}{\sqrt{2\hat{a}\hat{a}^\dagger}}
$$
And then one defines:
$$
    M = (\gamma_\alpha CS-\gamma_\beta SC')\otimes(CS^\dagger-S^\dagger C')
$$

In [3]:
a = p.a
ad = p.ad


M = tensor((ga*C*S - gb*S*Cp), (C*S.dag() - S.dag()*Cp))
# The second term of M is null
np.all(M.full() == 0)

AttributeError: 'Physics' object has no attribute 'a'

In [ ]:
def dissipator(X: Qobj, operator: Qobj):
    return X.dag() * operator * X - 1/2 * commutator(X.dag()*X, operator, kind='anti')

## Operators Evolution
$$
\begin{split}
    \Delta\hat{O}_n= \bigg[& \;
    \gamma_\alpha\left(
        \tfrac{1}{2}\mathcal{D}'[\hat{C}\otimes\hat{C} - 2\hat{S}\otimes\hat{S}^\dagger]
        + \mathcal{D}'[\hat{C}\otimes\hat{S} + \hat{S}\otimes\hat{C}']
    \right) \\[0.6em]
    &+ \gamma_\beta\left( 
        \tfrac{1}{2}\mathcal{D}'[\hat{C}'\otimes\hat{C}' - 2\hat{S}^\dagger\otimes\hat{S}] 
        + \mathcal{D}'[\hat{C}'\otimes\hat{S}^\dagger + \hat{S}^\dagger\otimes\hat{C}]
    \right) 
    \bigg]\hat{O}_n\\[0.6em]
    &- \frac{1}{2}\left[M+M^\dagger; \hat{O}_n\right]_+
\end{split}
$$

In [4]:
def evolve(operator):
    first_line = ga * (0.5*dissipator(tensor(C, C) - 2*tensor(S, S.dag()), operator) + 
                       dissipator(tensor(C, S) + tensor(S, Cp), operator))
    second_line = gb * (0.5*dissipator(tensor(Cp, Cp) - 2*tensor(S.dag(), S), operator) + 
                       dissipator(tensor(Cp, S.dag()) + tensor(S.dag(), C), operator))
    return operator + first_line + second_line

In [5]:
a_ = tensor(a, qeye(D))
ad_ = tensor(ad, qeye(D))
b_ = tensor(qeye(D), a)
bd_ = tensor(qeye(D), ad)
class CovarianceMatrix:
    operators = [
        [a_*a_, 0.5*(a_*ad_ + ad_*a_), 0.5*(a_*b_ + b_*a_), 0.5*(a_*bd_ + bd_*a_)]
    ]
s = CovarianceMatrix()
for o in [op for row in s.operators for op in row]:
    evolve(o)

NameError: name 'a' is not defined

In [7]:
op = []
op.append(tensor(ada, qeye(D)))
for i in range(TIMESTEPS):
    op.append(evolve(op[i]))

In [8]:
display(op[0])
display(op[-1])

Quantum object: dims = [[20, 20], [20, 20]], shape = (400, 400), type = oper, isherm = True
Qobj data =
[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 19.  0.  0.]
 [ 0.  0.  0. ...  0. 19.  0.]
 [ 0.  0.  0. ...  0.  0. 19.]]

Quantum object: dims = [[20, 20], [20, 20]], shape = (400, 400), type = oper, isherm = False
Qobj data =
[[14.72170298  0.          0.         ...  0.          0.
   0.        ]
 [ 0.         14.7244652   0.         ...  0.          0.
   0.        ]
 [ 0.          0.         14.72670799 ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ... 18.09273233  0.
   0.        ]
 [ 0.          0.          0.         ...  0.         18.09273384
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  18.09273476]]

### First Cavity

In [ ]:
display(op[0].ptrace(0))
display(op[-1].ptrace(0))

In [ ]:
# y = [np.real(o[0, 0]) / np.real(o[1, 1]) for o in op]
y = [np.real(o[0, 0]) for o in op]

In [ ]:
#x = np.linspace(0, 1, TIMESTEPS+1)
x = range(TIMESTEPS + 1)
plt.plot(x, y)
plt.show()